In [25]:
import xarray as xr
import xesmf
import matplotlib.pyplot as plt
import bottleneck
import numpy as np
import subprocess as sp
import os
import glob
import cartopy.crs as ccrs

# Create OBC files using MOM6-based boundary conditions

This example uses monthly output from an OM4_025-based simulation in order to generate
boundary conditions for a Pan-Arctic configuration. Ocean current vectors from a staggered
C-grid are co-located at q-points (corners). The grid rotation angles contained in the 
MOM6 supergrid files are used to align the currents with the target regional grid.

### Required files
    * Parent model supergrid file
    * Region model supergrid file and bathymetry
    * Parent model data

### Step 1: Open parent and regional model grids

In [26]:
def open_grid(path,decode_times=False):
    """Return a grid object containing staggered grid locations"""
    grid={}
    grid['ds']=xr.open_dataset(path,decode_times=False)
    grid['ds']=grid['ds'].drop_dims(['ny','nx'])
    grid['ds']=grid['ds'].drop_vars(['tile'])
    grid['nyp']=grid['ds'].nyp.data[-1]+1
    grid['nxp']=grid['ds'].nxp.data[-1]+1
    nxp=grid['nxp'];nyp=grid['nyp']
    grid['h'] = grid['ds'].isel(nxp=slice(1,nxp+1,2),nyp=slice(1,nyp+1,2))
    #The q grid is not symmetric, but Cu and Cv are
    grid['q'] = grid['ds'].isel(nxp=slice(2,nxp+1,2),nyp=slice(2,nyp+1,2))
    grid['Cu'] = grid['ds'].isel(nxp=slice(0,nxp+1,2),nyp=slice(1,nyp+1,2))
    grid['Cv'] = grid['ds'].isel(nxp=slice(1,nxp+1,2),nyp=slice(0,nyp+1,2))
    return grid

In [27]:
#Note that parent grid uv values are symmetric
path_parent_grid='/net2/mjh/ipynb/OM4_025/c192_mosaic/ocean_hgrid.nc'
parent_grid=open_grid(path_parent_grid)
path_regional_grid='./ocean_hgrid.nc'
regional_grid=open_grid(path_regional_grid)

### Step 2: Open parent model data and merge respective grids

In [28]:
def open_dataset(path,fields,grid):
    ds=xr.open_dataset(path,decode_times=False)
    
    tracer_list=[];uv_list=[]
    for f in fields:
        for fnam,val in zip(f.keys(),f.values()):
            if val=='h':tracer_list.append(fnam)
            if val=='Cu':uv_list.append(fnam)
            if val=='Cv':uv_list.append(fnam)
                
    ds_tr = xr.merge([ds, grid['h']])
    ds_u= xr.merge([ds,grid['Cu']])
    ds_v= xr.merge([ds,grid['Cv']])
    return {'ds_tr':ds_tr,'ds_u':ds_u,'ds_v':ds_v,'tracers':tracer_list,'uv':uv_list}

In [29]:
path_model_data='../data/20120101.ocean_month.nc'
fields=[{'temp':'h'},{'salt':'h'},{'ssh':'h'},{'u':'Cu'},{'v':'Cv'}]
model_data = open_dataset(path_model_data,fields,parent_grid)

### Remap velocities to q corners

In [30]:
def velocity_at_corners(ds_u,ds_v):
    nxp=ds_u.nxp[-1].data+1;nyp=ds_v.nyp[-1].data+1
    #upper-right q points
    u_q=0.5*(ds_u.u+ds_u.u.roll(roll_coords='yh',yh=-1)).isel(xq=slice(1,nxp))
    #upper-right q points
    v_q=0.5*(ds_v.v+ds_v.v.roll(roll_coords='xh',xh=-1)).isel(yq=slice(1,nyp))
    ds_uvq = xr.Dataset({'u':u_q,'v':v_q},coords={'time':ds_u.time,'lon':parent_grid['q'].x,'lat':parent_grid['q'].y,'angle_dx':parent_grid['q'].angle_dx})
    return ds_uvq

### Calculate corner velocities for a single time-slice

In [31]:

ds_u=model_data['ds_u'];ds_v=model_data['ds_v']
model_data['ds_uv']=velocity_at_corners(ds_u.isel(time=slice(0,1)),ds_v.isel(time=slice(0,1)))

In [ ]:
model_data['ds_uv']

In [32]:
def apply_rotation(u,v,angle_dx,time_slice=slice(0)):
    """Rotate from model space to easterly coordinates"""
    deg_rad=np.pi/180.    
    
    if time_slice is not None:
        t=u.time.isel(time=time_slice)
    else:
        t=u.time
        
    if time_slice is not None:
        ue=np.cos(angle_dx.data*deg_rad)*u.isel(time=time_slice).data-np.sin(angle_dx.data*deg_rad)*v.isel(time=time_slice).data
        vn=np.sin(angle_dx.data*deg_rad)*u.isel(time=time_slice).data+np.cos(angle_dx.data*deg_rad)*v.isel(time=time_slice).data  
    else:
        ue=np.cos(angle_dx.data*deg_rad)*u.data-np.sin(angle_dx.data*deg_rad)*v.data
        vn=np.sin(angle_dx.data*deg_rad)*u.data+np.cos(angle_dx.data*deg_rad)*v.data  
        
    us=ue.shape
    nyp=us[2];nxp=us[3]
    ue=xr.DataArray(ue,coords={'i':np.arange(1,nxp+1),'j':np.arange(1,nyp+1),'time':t,'z_l':u.z_l},dims=('time','z_l','j','i'))
    vn=xr.DataArray(vn,coords={'i':np.arange(1,nxp+1),'j':np.arange(1,nyp+1),'time':t,'z_l':u.z_l},dims=('time','z_l','j','i'))
        

    return ue,vn


In [33]:
def apply_rotation_transpose(ue,vn,angle_dx,time_slice=slice(0)):
    """Rotate from easterly coordinates to model space"""
    deg_rad=np.pi/180.
    
    if time_slice is not None:
        t=ue.time.isel(time=time_slice)
    else:
        t=ue.time

    if time_slice is not None:
        u=np.cos(angle_dx.data*deg_rad)*ue.isel(time=time_slice).data+np.sin(angle_dx.data*deg_rad)*vn.isel(time=time_slice).data
        v=-np.sin(angle_dx.data*deg_rad)*ue.isel(time=time_slice).data+np.cos(angle_dx.data*deg_rad)*vn.isel(time=time_slice).data
    else:
        u=np.cos(angle_dx.data*deg_rad)*ue.data+np.sin(angle_dx.data*deg_rad)*vn.data
        v=-np.sin(angle_dx.data*deg_rad)*ue.data+np.cos(angle_dx.data*deg_rad)*vn.data
    nyp=u.shape[2]
      
    u=xr.DataArray(u,coords={'i':np.arange(1,nyp+1),'time':t,'z_l':ue.z_l},dims=('time','z_l','i'))
    v=xr.DataArray(v,coords={'i':np.arange(1,nyp+1),'time':t,'z_l':ue.z_l},dims=('time','z_l','i'))


        
    return u,v

### Rotate currents

In [34]:

ds_uvq=model_data['ds_uv']
uq_rot,vq_rot= apply_rotation(ds_uvq.u,ds_uvq.v,ds_uvq.angle_dx,time_slice=None)
ds_uvq_r = xr.Dataset({'u':uq_rot,'v':vq_rot},coords={'time':ds_uvq.time,'lon':ds_uvq.lon,'lat':ds_uvq.lat})
model_data['ds_uv_r']=ds_uvq_r

In [ ]:
ds_uvq_r.u.isel(z_l=0).plot(vmin=-.2,vmax=.2,cmap='bwr')
title=plt.title('True zonal current')

In [35]:
def uvmag(u,v,z_slice=0):
    spd=np.sqrt(np.squeeze(u.isel(z_l=z_slice).data**2.0 + v.isel(z_l=z_slice).data**2))
    return xr.DataArray(spd,coords={'i':u.i.data,'j':u.j.data},dims=('j','i'))
    

In [ ]:
uvmag_r = uvmag(ds_uvq_r.u,ds_uvq_r.v)
plt.pcolormesh(parent_grid['q'].x,parent_grid['q'].y,uvmag_r.data,vmin=0.001,vmax=0.2)
plt.xlim(-120,60)
plt.ylim(50,90)
title=plt.title('Rotated current magnituge')

In [15]:
ds_regional=regional_grid['ds']
north = xr.Dataset()
north['lon'] = ds_regional['x'].isel(nyp=-1)
north['lat'] = ds_regional['y'].isel(nyp=-1)

# southern boundary
south = xr.Dataset()
south['lon'] = ds_regional['x'].isel(nyp=0)
south['lat'] = ds_regional['y'].isel(nyp=0)

# western boundary
west = xr.Dataset()
west['lon'] = ds_regional['x'].isel(nxp=0)
west['lat'] = ds_regional['y'].isel(nxp=0)

# eastern boundary
east = xr.Dataset()
east['lon'] = ds_regional['x'].isel(nxp=-1)
east['lat'] = ds_regional['y'].isel(nxp=-1)

In [16]:
ds_tr=model_data['ds_tr']
ds_tr = xr.Dataset({'temp':ds_tr.temp,'salt':ds_tr.salt},coords={'lon':ds_tr.x,'lat':ds_tr.y})

In [ ]:
regrid_north_uv = xesmf.Regridder(ds_uvq_r, north, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_north_uv.nc')
regrid_south_uv = xesmf.Regridder(ds_uvq_r, south, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_south_uv.nc')
regrid_east_uv = xesmf.Regridder(ds_uvq_r, east, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_east_uv.nc')
regrid_west_uv = xesmf.Regridder(ds_uvq_r, west, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_west_uv.nc')

In [ ]:
regrid_north_tr = xesmf.Regridder(ds_tr, north, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_north_tr.nc')
regrid_south_tr = xesmf.Regridder(ds_tr, south, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_south_tr.nc')
regrid_west_tr = xesmf.Regridder(ds_tr, west, 'nearest_s2d', 
                               locstream_out=True, periodic=False, filename='regrid_west_tr.nc')

In [19]:
ntime=ds_tr.temp.shape[0]
for i in range(ntime):
    uq_rot,vq_rot= apply_rotation(ds_uvq.u,ds_uvq.v,ds_uvq.angle_dx,time_slice=slice(i,i+1))
    ds_uvq_r = xr.Dataset({'u':uq_rot,'v':vq_rot},coords={'lon':ds_uvq.lon,'lat':ds_uvq.lat})  
    u_north_r = regrid_north_uv(ds_uvq_r['u'])
    v_north_r = regrid_north_uv(ds_uvq_r['v'])
    u_south_r = regrid_south_uv(ds_uvq_r['u'])
    v_south_r = regrid_south_uv(ds_uvq_r['v'])
    u_west_r = regrid_west_uv(ds_uvq_r['u'])
    v_west_r = regrid_west_uv(ds_uvq_r['v'])
    u_north,v_north=apply_rotation_transpose(u_north_r,v_north_r,ds_regional.angle_dx.isel(nyp=ds_regional.nyp[-1]),time_slice=None)
    ds_uv_north = xr.Dataset({'u':u_north,'v':v_north},coords={'lon':north.lon,'lat':north.lat,'z_l':ds_uvq.z_l})
    ds_uv_north.time.attrs['calendar']='gregorian'
    fnam='uv_north_'+str(i).zfill(2)+'.nc'
    ds_uv_north.to_netcdf(fnam,unlimited_dims='time',format='NETCDF3_CLASSIC')
    u_south,v_south=apply_rotation_transpose(u_south_r,v_south_r,ds_regional.angle_dx.isel(nyp=ds_regional.nyp[0]),time_slice=None)
    ds_uv_south = xr.Dataset({'u':u_south,'v':v_south},coords={'lon':south.lon,'lat':south.lat,'z_l':ds_uvq.z_l})
    fnam='uv_south_'+str(i).zfill(2)+'.nc'
    ds_uv_south.time.attrs['calendar']='gregorian'
    ds_uv_south.to_netcdf(fnam,unlimited_dims='time',format='NETCDF3_CLASSIC')
    u_west,v_west=apply_rotation_transpose(u_west_r,v_west_r,ds_regional.angle_dx.isel(nxp=ds_regional.nxp[0]),time_slice=None)
    ds_uv_west = xr.Dataset({'u':u_west,'v':v_west},coords={'lon':west.lon,'lat':west.lat,'z_l':ds_uvq.z_l})
    fnam='uv_west_'+str(i).zfill(2)+'.nc'
    ds_uv_west.time.attrs['calendar']='gregorian'
    ds_uv_west.to_netcdf(fnam,unlimited_dims='time',format='NETCDF3_CLASSIC')

In [ ]:
sp.run(['nccatm','uv_north_*.nc'],shell=True,check=True)
sp.run(['nccatm','uv_souh_*.nc'],shell=True,check=True)
sp.run(['nccatm','uv_west_*.nc'],shell=True,check=True)

In [ ]:
sp.run(['mv','uv_north_00.nc','uv_north.nc'])
sp.run(['mv','uv_south_00.nc','uv_south.nc'])
sp.run(['mv','uv_west_00.nc','uv_west.nc'])


In [23]:
for fl in glob.glob('uv_*_*.nc'):
    os.remove(fl)

In [145]:
ds_uv_south=xr.open_dataset('uv_south.nc')

In [ ]:
fig=plt.figure(1)
ds_uv_south.v.isel(time=6).plot(yincrease=False,vmin=-.1,vmax=.1,cmap='bwr')
plt.ylim(5000,0)


In [24]:
for i in range(ntime):
    temp_north = regrid_north_tr(ds_tr['temp'])
    salt_north = regrid_north_tr(ds_tr['salt'])
    ds_tr_north = xr.Dataset({'temp':temp_north,'salt':salt_north})
    ds_tr_north.time.attrs['calendar']='gregorian'
    fnam='tracers_north_'+str(i).zfill(2)+'.nc'
    ds_tr_north.to_netcdf(fnam,unlimited_dims='time',format='NETCDF3_CLASSIC')
    temp_south = regrid_south_tr(ds_tr['temp'])
    salt_south = regrid_south_tr(ds_tr['salt'])
    ds_tr_south = xr.Dataset({'temp':temp_south,'salt':salt_south})
    ds_tr_south.time.attrs['calendar']='gregorian'
    fnam='tracers_south_'+str(i).zfill(2)+'.nc'
    ds_tr_south.to_netcdf(fnam,unlimited_dims='time',format='NETCDF3_CLASSIC')
    temp_west = regrid_west_tr(ds_tr['temp'])
    salt_west = regrid_west_tr(ds_tr['salt'])
    ds_tr_west = xr.Dataset({'temp':temp_west,'salt':salt_west})
    ds_tr_west.time.attrs['calendar']='gregorian'
    fnam='tracers_west_'+str(i).zfill(2)+'.nc'
    ds_tr_west.to_netcdf(fnam,unlimited_dims='time',format='NETCDF3_CLASSIC')
    

In [ ]:
sp.run(['nccatm','tracers_north_*.nc'],shell=True,check=True)
sp.run(['mv','tracers_north_00.nc','tracers_north.nc'])
sp.run(['nccatm','tracers_north_*.nc'],shell=True,check=True)
sp.run(['mv','tracers_south_00.nc','tracers_south.nc'])
sp.run(['nccatm','tracers_west_*.nc'],shell=True,check=True)
sp.run(['mv','tracers_west_00.nc','tracers_west.nc'])

In [192]:
for fl in glob.glob('tracers_*_*.nc'):
    os.remove(fl)

In [ ]:
fig=ds_tr_north.isel(time=2).plot(figsize=[8, 6], yincrease=False, cmap='jet')
plt.ylim(200,0)

In [321]:
ds_n=xr.open_dataset('tracers_north.nc')
ds_s=xr.open_dataset('tracers_south.nc')
ds_w=xr.open_dataset('tracers_west.nc')

In [344]:
nps = ccrs.Stereographic(true_scale_latitude=60., central_latitude=90.,central_longitude=0.)
bering= ccrs.LambertConformal(central_latitude = 60, 
                             central_longitude = -140, 
                             standard_parallels = (60, 60))
barents= ccrs.LambertConformal(central_latitude = 50, 
                             central_longitude = -30, 
                             standard_parallels = (60, 60))

In [447]:
dsr_topo=xr.open_dataset('bathy.nc')
dsr_topo=dsr_topo.drop_vars(['x','y','max','min','std','count'])
dsr_topo=dsr_topo.rename_dims({'x':'nx','y':'ny'})
dsr_topo=dsr_topo.rename({'mean':'elevation'})

In [448]:
dsr_topo = xr.merge([dsr_topo, regional_grid['h']])

In [ ]:
dsr_topo

In [ ]:
dsr_topo.elevation.plot(vmax=0.,vmin=-1000.,cmap='jet')

In [ ]:
ax = plt.axes(projection=bering)
ax.coastlines()
extent = [-190,-120, 50,70]
ax.set_extent(extent)
lons=north.lon.data
lons[lons>180]=lons[lons>180]-360.
ax.pcolormesh(dsr_topo.x,dsr_topo.y,np.ma.masked_where(dsr_topo.elevation>0,-dsr_topo.elevation),vmin=0,vmax=500.,transform=ccrs.PlateCarree(),cmap='jet')
ax.plot(lons, north.lat, transform = ccrs.PlateCarree(),color='k')
lons=west.lon.data
lons[lons>0]=lons[lons>0]-360.
ax.plot(lons, west.lat, transform = ccrs.PlateCarree(),color='k')
lons=east.lon.data
lons[lons>180]=lons[lons>180]-360.
ax.plot(lons, east.lat, transform = ccrs.PlateCarree(),color='k')

In [ ]:
ax = plt.axes(projection=barents)
ax.coastlines()
#extent = [-190,-120, 50,70]
extent = [-80,30, 40,90]
#extent = [-180,180, 40,90]
ax.set_extent(extent)
lons=south.lon.data
lons[lons>180]=lons[lons>180]-360.
ax.pcolormesh(dsr_topo.x,dsr_topo.y,np.ma.masked_where(dsr_topo.elevation>0,-dsr_topo.elevation),vmin=0,vmax=500.,transform=ccrs.PlateCarree(),cmap='jet')
ax.plot(lons, south.lat, transform = ccrs.PlateCarree(),color='k')
lons=west.lon.data
lons[lons>0]=lons[lons>0]-360.
ax.plot(lons, west.lat, transform = ccrs.PlateCarree(),color='k')
lons=east.lon.data
lons[lons>180]=lons[lons>180]-360.
ax.plot(lons, east.lat, transform = ccrs.PlateCarree(),color='k')
lons=north.lon.data
lons[lons>180]=lons[lons>180]-360.
ax.plot(lons, north.lat, transform = ccrs.PlateCarree(),color='k')